In [ ]:
!pip install indic-nlp-library
!pip install nlp_utils
!pip install contractions
!pip3 install contractions
import pandas as pd
import numpy as np
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from matplotlib import rcParams
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
import re
import nltk
import string
import collections
import nlp_utils as nu
from collections import Counter
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.tokenize import word_tokenize,sent_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from indicnlp.tokenize import indic_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import metrics
from sklearn.metrics import accuracy_score, auc, confusion_matrix, roc_auc_score, roc_curve, recall_score, precision_score, recall_score, f1_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
tstdf = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/PR/Banglish_Balanced.xlsx')

tstdf['Class'] = tstdf['Class'].replace('Others', 1)
tstdf['Class'] = tstdf['Class'].replace('Sexual', 2)
tstdf['Class'] = tstdf['Class'].replace('Slang', 3)
tstdf

,Text,Class
0,Kanki ki der Allah sob samoy valo rake,1
1,Ahare gali dite mon chay parlamna,1
2,Vai toder kaj kam ase? Ke ki vabe pad dilo tao...,1
3,Koi taka danda korchos,1
4,Er pasay Dim Therapy Dawa hok,1
...,...,...
1670,iye gula choto lagca kno,2
1671,"sexi Good fegar, amr hoba tumi",2
1672,Khelte peebe na thik thak vabe,2
1673,DuDu boro hoye gese di apner unar hand er chap...,2


In [ ]:
!pip install pandas googletrans==4.0.0-rc1
from googletrans import Translator
translator = Translator()

def remove_punctuation(text):
    translator = str.maketrans(string.punctuation, ' ' * len(string.punctuation))
    text_without_punctuation = text.translate(translator)
    return text_without_punctuation

def tokenize_bangla_transliterated(text):
    tokens = indic_tokenize.trivial_tokenize(text)
    return ' '.join(tokens)

# Translate Bangla transliterated text to proper Bangla
translator = Translator()

def translate_text(text):
    try:
        # Specify the source language as 'bn' (Bangla) for transliterated text
        translation = translator.translate(text, src='en', dest='bn')
        print(f"Translated Text: {translation}")
        return translation.text
    except Exception as e:
        print(f"Translation error: {e}")
        return text

tstdf['Text'] = tstdf['Text'].apply(remove_punctuation)
tstdf['Bangla_Text'] = tstdf['Text'].apply(translate_text)
data = tstdf['Bangla_Text'].tolist()
# data = tstdf['Text'].tolist()
df = pd.DataFrame(data)
df['Tokenized_Text'] = tstdf['Bangla_Text'].apply(tokenize_bangla_transliterated)
# df['Tokenized_Text'] = tstdf['Text'].apply(tokenize_bangla_transliterated)
df['Class'] = tstdf['Class']

df['Tokenized_Text'].dropna(inplace=True)
df.dropna(subset=['Tokenized_Text'], inplace=True)

Translated Text: Translated(src=en, dest=bn, text=কঙ্কি কি ডের আল্লাহ সোব সাময় ভালো রাকে, pronunciation=Kaṅki ki ḍēra āllāha sōba sāmaẏa bhālō rākē, extra_data="{'confiden...")
Translated Text: Translated(src=en, dest=bn, text=আহরে গালি ডাইট সোম চ্যা পার্লামনা, pronunciation=Āharē gāli ḍā'iṭa sōma cyā pārlāmanā, extra_data="{'confiden...")
Translated Text: Translated(src=en, dest=bn, text=ভাই টডার কাজ কাম এএস কে কি কি ভাবে প্যাড দিলো তাও জান্তে হোবে টডার, pronunciation=Bhā'i ṭaḍāra kāja kāma ē'ēsa kē ki ki bhābē pyāḍa dilō tā'ō jāntē hōbē ṭaḍāra, extra_data="{'confiden...")
Translated Text: Translated(src=en, dest=bn, text=কোয়ে তাকা ডান্ডা করচোস, pronunciation=Kōẏē tākā ḍānḍā karacōsa, extra_data="{'confiden...")
Translated Text: Translated(src=en, dest=bn, text=এর পাসে ম্লান থেরাপি দাওয়া হক, pronunciation=Ēra pāsē mlāna thērāpi dā'ōẏā haka, extra_data="{'confiden...")
Translated Text: Translated(src=en, dest=bn, text=তুই ই দিন ই অ্যাশকোস ওভিশাব হোয়া, pronunciation=Tu'i i dina i ay

In [ ]:
print(df['Tokenized_Text'])
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['Tokenized_Text'])
Y = df['Class']

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,random_state=42)
print(X_train.shape,X_test.shape,Y_train.shape,Y_test.shape)

0                 কঙ্কি কি ডের আল্লাহ সোব সাময় ভালো রাকে
1                       আহরে গালি ডাইট সোম চ্যা পার্লামনা
2       ভাই টডার কাজ কাম এএস কে কি কি ভাবে প্যাড দিলো ...
3                                 কোয়ে তাকা ডান্ডা করচোস
4                          এর পাসে ম্লান থেরাপি দাওয়া হক
                              ...                        
1670                             iye gula choto lagca kno
1671                        sexi Good fegar amr hoba tumi
1672                       Khelte peebe na thik thak vabe
1673    DuDu boro hoye gese di apner unar hand er chap...
1674    jei thanda ekhon eto choto kapor kno dekha jay to
Name: Tokenized_Text, Length: 1675, dtype: object
(1340, 2897) (335, 2897) (1340,) (335,)


In [ ]:
log_reg = LogisticRegression().fit(X_train, Y_train)

#predict on train
train_preds = log_reg.predict(X_train)
precision_train = precision_score(Y_train, train_preds, average='weighted')
recall_train = recall_score(Y_train, train_preds, average='weighted')
f1_train = f1_score(Y_train, train_preds, average='weighted')
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds))
print("Precision on train is:", precision_train)
print("Recall on train is:", recall_train)
print("F1 Score on train is:", f1_train)
print("\n")

#predict on test
test_preds = log_reg.predict(X_test)
precision_test = precision_score(Y_test, test_preds, average='weighted')
recall_test = recall_score(Y_test, test_preds, average='weighted')
f1_test = f1_score(Y_test, test_preds, average='weighted')
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds))
print("Precision on test is:", precision_test)
print("Recall on test is:", recall_test)
print("F1 Score on test is:", f1_test)

Model accuracy on train is:  0.8544776119402985
Precision on train is: 0.8603202422220271
Recall on train is: 0.8544776119402985
F1 Score on train is: 0.8542937225051853


Model accuracy on test is:  0.5014925373134328
Precision on test is: 0.5213719862227325
Recall on test is: 0.5014925373134328
F1 Score on test is: 0.4914110764857034


In [ ]:
#fit the model on train data
SVM = SVC(kernel='linear')
SVM.fit(X_train, Y_train)

#predict on train
train_preds5 = SVM.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds5))

#predict on test
test_preds5 = SVM.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds5))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds5))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds5))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds5).sum(),'/',((Y_test == test_preds5).sum()+(Y_test != test_preds5).sum()))
print('-'*50)

precision_test = precision_score(Y_test, test_preds, average='weighted')
recall_test = recall_score(Y_test, test_preds, average='weighted')
f1_test = f1_score(Y_test, test_preds, average='weighted')

print("Precision (Test):", precision_test)
print("Recall (Test):", recall_test)
print("F1 Score (Test):", f1_test)

Model accuracy on train is:  0.8529850746268657
Model accuracy on test is:  0.4925373134328358
--------------------------------------------------
confusion_matrix train is:  [[408  14  39]
 [ 49 314  45]
 [ 41   9 421]]
confusion_matrix test is:  [[52 23 54]
 [23 40 39]
 [18 13 73]]
Wrong predictions out of total
--------------------------------------------------
170 / 335
--------------------------------------------------
Precision (Test): 0.5213719862227325
Recall (Test): 0.5014925373134328
F1 Score (Test): 0.4914110764857034


In [ ]:
DT = DecisionTreeClassifier().fit(X,Y)

#predict on train
train_preds2 = DT.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds2))

#predict on test
test_preds2 = DT.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds2))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds2))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds2))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds2).sum(),'/',((Y_test == test_preds2).sum()+(Y_test != test_preds2).sum()))
print('-'*50)

# Kappa Score
print('KappaScore is: ', metrics.cohen_kappa_score(Y_test,test_preds2))

Model accuracy on train is:  0.95
Model accuracy on test is:  0.9044776119402985
--------------------------------------------------
confusion_matrix train is:  [[436   3  22]
 [  8 378  22]
 [  7   5 459]]
confusion_matrix test is:  [[118   0  11]
 [  6  84  12]
 [  3   0 101]]
Wrong predictions out of total
--------------------------------------------------
32 / 335
--------------------------------------------------
KappaScore is:  0.8558713598106967


In [ ]:
#fit the model on train data
RF=RandomForestClassifier().fit(X_train,Y_train)
#predict on train
train_preds3 = RF.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds3))

#predict on test
test_preds3 = RF.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds3))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds3))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds3))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds3).sum(),'/',((Y_test == test_preds3).sum()+(Y_test != test_preds3).sum()))
print('-'*50)

Model accuracy on train is:  0.9537313432835821
Model accuracy on test is:  0.47761194029850745
--------------------------------------------------
confusion_matrix train is:  [[433   4  24]
 [  5 379  24]
 [  3   2 466]]
confusion_matrix test is:  [[43 27 59]
 [17 47 38]
 [20 14 70]]
Wrong predictions out of total
--------------------------------------------------
175 / 335
--------------------------------------------------


In [ ]:
#fit the model on train data
KNN = KNeighborsClassifier().fit(X_train,Y_train)
#predict on train
train_preds4 = KNN.predict(X_train)
#accuracy on train
print("Model accuracy on train is: ", accuracy_score(Y_train, train_preds4))

#predict on test
test_preds4 = KNN.predict(X_test)
#accuracy on test
print("Model accuracy on test is: ", accuracy_score(Y_test, test_preds4))
print('-'*50)

#Confusion matrix
print("confusion_matrix train is: ", confusion_matrix(Y_train, train_preds4))
print("confusion_matrix test is: ", confusion_matrix(Y_test, test_preds4))
print('Wrong predictions out of total')
print('-'*50)

# Wrong Predictions made.
print((Y_test !=test_preds4).sum(),'/',((Y_test == test_preds4).sum()+(Y_test != test_preds4).sum()))
print('-'*50)

precision_test = precision_score(Y_test, test_preds, average='weighted')
recall_test = recall_score(Y_test, test_preds, average='weighted')
f1_test = f1_score(Y_test, test_preds, average='weighted')

print("Precision (Test):", precision_test)
print("Recall (Test):", recall_test)
print("F1 Score (Test):", f1_test)

Model accuracy on train is:  0.6037313432835821
Model accuracy on test is:  0.30149253731343284
--------------------------------------------------
confusion_matrix train is:  [[130 303  28]
 [ 20 373  15]
 [ 27 138 306]]
confusion_matrix test is:  [[  8 103  18]
 [  9  84   9]
 [ 10  85   9]]
Wrong predictions out of total
--------------------------------------------------
234 / 335
--------------------------------------------------
Precision (Test): 0.5213719862227325
Recall (Test): 0.5014925373134328
F1 Score (Test): 0.4914110764857034


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report

X = df['Tokenized_Text']
y = df['Class']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Vectorize the Bangla text using CountVectorizer
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# Initialize Multinomial Naive Bayes model
nb_model = MultinomialNB()

# Train the model
nb_model.fit(X_train_vectorized, y_train)

# Make predictions on the test set
y_pred = nb_model.predict(X_test_vectorized)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

print(f'Accuracy: {accuracy}')
print('Classification Report:\n', classification_rep)


Accuracy: 0.5104477611940299
Classification Report:
               precision    recall  f1-score   support

           1       0.54      0.59      0.57       129
           2       0.52      0.36      0.43       102
           3       0.47      0.56      0.51       104

    accuracy                           0.51       335
   macro avg       0.51      0.50      0.50       335
weighted avg       0.51      0.51      0.51       335

